In [ ]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [1]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing

In [2]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [3]:
import json
import argparse
import lxml
import config

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

In [ ]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


In [ ]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



In [ ]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [ ]:
os.getcwd()


In [4]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [5]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df

,int_id,int_nctid,int_type,int_name,int_description
0,4547568,NCT01340339,Device,Phototherapy,Fluorescent reverse phototherapy (7 white ligh...
1,4547569,NCT01340339,Device,Phototherapy,super LED reverse phototherapy (17 bulbs arran...
2,4547570,NCT01340365,Behavioral,Tai Chi Exercise,Practicing Tai Chi exercise 4 times a week for...
3,4547571,NCT01340365,Behavioral,Tai Chi,"Usual care, individuals attend testing session..."
4,4547572,NCT01340391,Device,Omnicast,dorsal splint 2-5 weeks
...,...,...,...,...,...
685271,4139681,NCT03104712,Other,Rice + Pesto,50g available carbohydrate
685272,4139682,NCT03104465,Behavioral,Mindfulness Training,interactive. web-based mindfulness training co...
685273,4139683,NCT03104361,Biological,Platelet-Rich Plasma,Patients who meet all eligible requirements fo...
685274,4139684,NCT03104283,Drug,Apatinib,take apatinib orally until disease progression...


In [6]:
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])

In [7]:
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df

,con_id,con_nctid,con_name,con_downcase_name
0,3072834,NCT03254264,Autism Spectrum Disorder,autism spectrum disorder
1,3072835,NCT03254329,Human Milk Nutrient Reference Values,human milk nutrient reference values
2,3349485,NCT03460652,ADHD,adhd
3,3349486,NCT03460899,Diabetes Mellitus With Hypoglycemia,diabetes mellitus with hypoglycemia
4,3072836,NCT03254342,Major Depressive Disorder,major depressive disorder
...,...,...,...,...
688783,3072052,NCT03261687,Pregnancy Related,pregnancy related
688784,3072053,NCT03261687,Pelvic Girdle Pain,pelvic girdle pain
688785,3072054,NCT03261622,Faecal Incontinence,faecal incontinence
688786,3072055,NCT03261622,Fecal Incontinence,fecal incontinence


In [8]:
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])

In [9]:
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df

,browseint_id,browseint_nctid,browseint_meshterm,browseint_meshterm_downcase,browseint_meshtype
0,4935968,NCT04327843,Haloperidol,haloperidol,mesh-list
1,4935969,NCT04327843,Haloperidol decanoate,haloperidol decanoate,mesh-list
2,4935970,NCT04327843,Antiemetics,antiemetics,mesh-ancestor
3,5198960,NCT00808223,Alefacept,alefacept,mesh-list
4,5198961,NCT00808223,Dermatologic Agents,dermatologic agents,mesh-ancestor
...,...,...,...,...,...
1335262,4400668,NCT03100786,"Antineoplastic Agents, Hormonal","antineoplastic agents, hormonal",mesh-ancestor
1335263,4400669,NCT03100786,Antineoplastic Agents,antineoplastic agents,mesh-ancestor
1335264,4400670,NCT03100565,Lidocaine,lidocaine,mesh-list
1335265,4400671,NCT03100565,"Anesthetics, Local","anesthetics, local",mesh-ancestor


In [10]:
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])

In [11]:
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df

,browsecon_id,browsecon_nctid,browsecon_meshterm,browsecon_meshterm_downcase,browsecon_meshtype
0,9631397,NCT04035564,Hyponatremia,hyponatremia,mesh-list
1,9631398,NCT04035564,Water-Electrolyte Imbalance,water-electrolyte imbalance,mesh-ancestor
2,9631399,NCT04035564,Metabolic Diseases,metabolic diseases,mesh-ancestor
3,9631400,NCT04032652,Colitis,colitis,mesh-list
4,9631401,NCT04032652,"Colitis, Ulcerative","colitis, ulcerative",mesh-list
...,...,...,...,...,...
2573556,8601072,NCT03213899,Disease Attributes,disease attributes,mesh-ancestor
2573557,8601073,NCT03213899,Pathologic Processes,pathologic processes,mesh-ancestor
2573558,8601074,NCT03214016,Hypertension,hypertension,mesh-list
2573559,8601075,NCT03214016,Vascular Diseases,vascular diseases,mesh-ancestor


In [12]:
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])

In [13]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [ ]:
conditions_df


In [14]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')

In [ ]:
df.shape

In [15]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)

In [ ]:
df_dedup.shape

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

In [ ]:
df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


# My own notes

Use pool/multiprocess to make faster
resolve multiple possible CURIEs per concept

In [16]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [ ]:
ct_extract

In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [17]:
final_df = pd.DataFrame(columns=['subject','predicate','object','relation','subject_name','object_name','category'])

In [18]:
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
final_df['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
final_df['predicate'] = 'biolink:related_to biolink:clinically_demonstrated_to biolink: hypothesized_to'
final_df['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
final_df['relation'] = 'RO:????'
final_df.subject_name = ct_extract.con_downcase_name
final_df.object_name = ct_extract.int_name
final_df.category = 'biolink:Association'
final_df['nctid'] = ct_extract['nctid']
final_df['nctid_curie'] = ct_extract['nctid_curie']


In [ ]:
final_df

In [ ]:
psoriasis = final_df[final_df['subject_name'].str.contains('psoriasis')]
psoriasis



In [ ]:
zinc = final_df[final_df['object_name'].str.contains('zinc', na=False)]
zinc

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
zinc

In [ ]:
psoriasis[['subject_name', 'object_name', 'nctid']].to_csv('psoriasis.csv', sep =',', index=False)
# zinc[['subject_name', 'object_name', 'nctid']].to_csv('zinc.csv', sep ='\t', index=False)


In [ ]:
psoriasis_zinc = psoriasis[psoriasis['object_name'].str.contains('zn')]

In [ ]:
psoriasis_zinc

In [ ]:
final_df.to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [19]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

# NOTE: YOU MUST START the SKR/Medpost Part-of-Speech Tagger Server and the Word Sense Disambiguation (WSD) Server IN TERMINAL

# % ./bin/skrmedpostctl start
# % ./bin/wsdserverctl start


# stop these servers afterwards



In [302]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python
../../../../../../../../Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin


In [20]:
mm = MetaMap.get_instance('/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/metamap18')

In [22]:
sents = ['myocardial infarction']  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

ConceptMMI(index='USER', mm='MMI', score='13.22', preferred_name='Myocardial Infarction', cui='C0027051', semtypes='[dsyn]', trigger='["Infarction, Myocardial"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='C14.280.647.500;C14.907.585.500')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Electrocardiogram: myocardial infarction (finding)', cui='C0428953', semtypes='[lbtr]', trigger='["MYOCARDIAL INFARCTION"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Myocardial Infarction ECG Assessment', cui='C3810814', semtypes='[diap]', trigger='["Myocardial Infarction"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Myocardial Infarction, CTCAE', cui='C4552959', semtypes='[fndg]', trigger='["Myocardial infarction"-tx-1-"myocardial inf

In [82]:
error

In [ ]:
# check char encoding, is ascii??
    
for ct_string in final_df['subject_name'][:40]:
#     print(type(ct_string))
    print(ct_string.isascii())
    

In [27]:
# MetaMap does not work on non-ascii char, so create dict with what non-ascii char should be replaced with

# diseases = list(set(list(final_df['subject_name'])))
ascii_char_dict = {",": ' ',
                   ';': ' ',
                   '；': ' ',
                   '≥': ' greater than or equal to ',
                   '≤': ' less than or equal to ',
                   '<': ' less than or equal to ',
                   '>': ' less than or equal to ',
                   'ç': 'c',
                   '®': ' ',
                   '©': ' ',
                   'ï': 'i',
                   'ї': 'i',
                   '̇': '',
                   '"̇': ' ',
#                    '"̇"': ' ',
                   '"': ' ',
                   '\"': ' ',
                   'é': 'e',
                   'è': 'e',
                   'í': 'i',
                   '₁': '1',
                   'ı': '1',
                   '，': ' ',
                   'β': ' beta ',
                   'ß': ' beta ',
                   'ö': 'o',
                   'ō': 'o',
                   '´': "",
                   '²': '2',
                   'γ': ' gamma ',
                   'α': ' alpha ',
                   'ü': 'u',
                   'μg': 'micrograms',
                   'µg': 'micrograms',
                   'µ': 'mu',
                   'μ': 'mu',
                   '•': ' ',
                   '°': ' degrees ',
                   'λ': ' lambda ',
                   'ﬂ': 'fl',
                   'ⅱ': '2',
                   'δ': ' delta ',
#                    'stage i': 'stage 1',
#                    'stage ii': 'stage 2',
#                    'stage iii': 'stage 3',
                   'ⅲ': '3',
                   '（': ' ',
                   '）': ' ',
                   '、': ' ',
                   '¨': ' ',
                   '™': ' ',
                   '：': ' ',
                   'œ': ' ',
                   '／': ' ',
                   '.': ' ',
                   '±': ' ',
                   'ő': 'o'
                   
                  }


In [23]:
diseases = list(set(list(final_df['subject_name'])))
interventions = list(filter(None, final_df['object_name'].unique()))

In [29]:
import re
non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")


def custom_make_translation(text, translation):
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text, ascii_char_dict)

In [30]:
non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")

import re

def custom_make_translation(text):
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text)

In [31]:
translated


'split-virion, non-adjuvanted H1N1 vaccine of 15  g'

In [32]:
diseases_translated = dict((i, custom_make_translation(i)) for i in diseases)
interventions_translated = dict((i, custom_make_translation(i)) for i in interventions)

In [33]:
diseases_translated = dict((i, custom_make_translation(i)) for i in diseases[:20])
interventions_translated = dict((i, custom_make_translation(i)) for i in interventions[:20])

In [34]:
interventions_translated

{'Early Start Denver Model': 'Early Start Denver Model',
 "'as-usual' intervention": "'as-usual' intervention",
 'Assessment of human milk nutrient composition': 'Assessment of human milk nutrient composition',
 'KP415 oral capsule': 'KP415 oral capsule',
 'Euglycaemic Clamp': 'Euglycaemic Clamp',
 'Hyperinsulinaemic/Hypoglycaemic Clamp': 'Hyperinsulinaemic/Hypoglycaemic Clamp',
 'VX-659/TEZ/IVA': 'VX-659/TEZ/IVA',
 'TEZ/IVA': 'TEZ/IVA',
 'IVA': 'IVA',
 'Placebo': 'Placebo',
 'MT-6548': 'MT-6548',
 'Onasemnogene Abeparvovec-xioi': 'Onasemnogene Abeparvovec-xioi',
 'Rimegepant': 'Rimegepant',
 '3D printed model': '3D printed model',
 'Verbal Counselling': 'Verbal Counselling',
 'Asymmetric STN-DBS': 'Asymmetric STN-DBS',
 'Methacetin Breath Test': 'Methacetin Breath Test',
 'Emricasan': 'Emricasan',
 'Placebo oral capsule': 'Placebo oral capsule',
 'High fiber sugar free biscuit': 'High fiber sugar free biscuit'}

In [173]:
# def run_metamap(input_term, source_restriction):
#     concepts_dict = dict()
#     try:
#         concepts,error = mm.extract_concepts(sentences = [input_term], restrict_to_sources = source_restriction)
#         concepts_dict[input_term] = concepts
#     except:
#         concepts_dict[input_term] = None 
#     return(concepts_dict)

In [46]:
def run_metamap_orig(input_term):
    concepts_dict = dict()
    try:
        concepts,error = mm.extract_concepts(sentences = [input_term])
        concepts_dict[input_term] = concepts
    except:
        concepts_dict[input_term] = None 
    return(concepts_dict)

In [71]:
def run_metamap(input_term, source_restriction):
    concepts_dict = dict()
    if all(x is None for x in source_restriction):
        try:
            concepts,error = mm.extract_concepts([input_term])
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    else:
        try:
            concepts,error = mm.extract_concepts([input_term], restrict_to_sources=source_restriction)
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    return(concepts_dict)

In [72]:
number_of_processes = 4
metamapped_diseases = multiprocessing.Pool(number_of_processes).starmap(run_metamap,
                                                                        zip(list(diseases_translated.values()), [['SNOMEDCT_US', 'SNOMEDCT_VET', 'ICD10CM', 'ICD10CM', 'ICD10PCS', 'ICD9CM', 'ICD9CM']]*len(list(diseases_translated.values()))))
metamapped_interventions = multiprocessing.Pool(number_of_processes).starmap(run_metamap, 
                                                                             zip(list(interventions_translated.values()),
                                                                                 [[None]]*len(list(interventions_translated.values()))))
                                                                             


In [ ]:
# link nct ids of studies that have the same disease and intervention pair---multiple studies may have studies the same intervention
# restrict returned CUIs of metamapped concepts???

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
mapped